<a href="https://colab.research.google.com/github/syeong1218/python/blob/master/%EA%B5%AC%ED%98%84_%EC%BD%94%EB%93%9C_%EC%84%A4%EB%AA%85%EC%A4%91_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import 할 것들
import os
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
smifile = "GDBChEMBL".smi"   # GDBChEMBL 의 SMILES 데이터 세트를 사용합니다.
data = pd.read_csv(smifile, delimiter = "\t", names = ["smiles","No","Int"])
from sklearn.cross_validation import train_test_split  #사이킷런을 import 합니다.
smiles_train, smiles_test = train_test_split(data["smiles"], random_state=42) 
print(smiles_train.shape) # 훈련데이터와 시험데이터를 출력합니다.
print(smiles_test.shape)

In [0]:
charset = set("".join(list(data.smiles))+"!E") 
char_to_int = dict((c,i) for i,c in enumerate(charset))
int_to_char = dict((i,c) for i,c in enumerate(charset))
embed = max([len(smile) for smile in data.smiles]) + 5
print str(charset)
print(len(charset), # RNN이 배치 모드에서 학습되고 최대 발생 + 일부 추가된 발생으로 설정되므로 SMILES 문자열의 최대 길이가 필요

In [0]:
def vectorize(smiles):  # 함수를 정의
        one_hot =  np.zeros((smiles.shape[0], embed , len(charset)),dtype=np.int8) #
        for i,smile in enumerate(smiles):
            #encode the startchar
            one_hot[i,0,char_to_int["!"]] = 1  # 원 핫 인코딩을 수행
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1  # 원 핫 인코딩을 수행
            #Encode endchar
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1  # 원 핫 인코딩을 수행
        #Return two, one for input and the other for output
        return one_hot[:,0:-1,:], one_hot[:,1:,:]
X_train, Y_train = vectorize(smiles_train.values) # X , Y 훈련 데이터를 벡터화
X_test,Y_test = vectorize(smiles_test.values) # X , Y 시험 데이터를 벡터화
print (smiles_train.iloc[0])  # 값을 출력
plt.matshow(X_train[0].T)
#print X_train.shape

In [0]:
# Keras 를 import함
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Concatenate
from keras import regularizers
input_shape = X_train.shape[1:]
output_dim = Y_train.shape[-1]
latent_dim = 64
lstm_dim = 64

In [0]:
unroll = False # RNN의 속도를 높이기 위해 False로 지정
encoder_inputs = Input(shape=input_shape)  # 인코더에 입력
encoder = LSTM(lstm_dim, return_state=True,
                unroll=unroll) # 위에서 lstm_dim이 64라고했으므로 64LSTM 셀의 단일층은 입력SMILES 문자열을 읽는 데 사용
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = Concatenate(axis=-1)([state_h, state_c])
neck = Dense(latent_dim, activation="relu")
neck_outputs = neck(states)

In [0]:
decode_h = Dense(lstm_dim, activation="relu")
decode_c = Dense(lstm_dim, activation="relu")
state_h_decoded =  decode_h(neck_outputs)
state_c_decoded =  decode_c(neck_outputs)
encoder_states = [state_h_decoded, state_c_decoded]
decoder_inputs = Input(shape=input_shape)
decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,  #각 sequence마다 출력값을 출력. LSTM 레이어를 여러개로 쌓아올릴 때는 return_sequence=True 옵션을 사용
                    unroll=unroll
                   )
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(output_dim, activation='softmax')  # Decoder 층에 마지막에 softmax 활성화함수 사용
decoder_outputs = decoder_dense(decoder_outputs)  # 출력
#두 위치에 대해 훈련 벡터를 입력하고 입력에 앞서 한 문자를 예측하는 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print model.summary()

In [0]:
from keras.callbacks import History, ReduceLROnPlateau # History : 케라스는 학습시킬 때 fit함수를 사용하고 History 객체가 반환됩니다. History는 이어지는 epoch 각각에서의 훈련 손실 값과 metric 값을 기록합니다.
h = History()
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=10, min_lr=0.000001, verbose=1, epsilon=1e-5)

In [0]:
import pickle
pickle.dump(h.history, file("Blog_history.pickle","w"))

In [0]:
plt.plot(h.history["loss"], label="Loss")
plt.plot(h.history["val_loss"], label="Val_Loss")
plt.yscale("log")
plt.legend()


In [0]:
# 아래 코드는 아무것도 출력하지 않으므로 테스트 세트에서 100 개의 테스트 된 SMILES가 완벽하게 재구성됩니다.
for i in range(100):
    v = model.predict([X_test[i:i+1], X_test[i:i+1]]) #Can't be done as output not necessarely 1 , 모델을 시험데이터로 예측
    idxs = np.argmax(v, axis=2)
    pred=  "".join([int_to_char[h] for h in idxs[0]])[:-1]
    idxs2 = np.argmax(X_test[i:i+1], axis=2)
    true =  "".join([int_to_char[k] for k in idxs2[0]])[1:]
    if true != pred:
        print true, pred

In [0]:
smiles_to_latent_model = Model(encoder_inputs, neck_outputs) # 숨어있는 모델 (은닉모델)

smiles_to_latent_model.save("Blog_simple_smi2lat.h5")

In [0]:
latent_input = Input(shape=(latent_dim,)) #은닉층 입력
#reuse_layers : 잠재 공간과 일치하는 새로운 입력이 정의되었지만 이전의 층을 재사용하여 h(hidden) 및 c(cell) 상태를 얻을 수 있습니다. 그렇게하면 가중치가 훈련 된 모델에서 상속됩니다.
state_h_decoded_2 =  decode_h(latent_input)
state_c_decoded_2 =  decode_c(latent_input)
latent_to_states_model = Model(latent_input, [state_h_decoded_2, state_c_decoded_2])
latent_to_states_model.save("Blog_simple_lat2state.h5")

In [0]:
#Last one is special, we need to change it to stateful, and change the input shape
inf_decoder_inputs = Input(batch_shape=(1, 1, input_shape[1]))
inf_decoder_lstm = LSTM(lstm_dim,
                    return_sequences=True,
                    unroll=unroll,
                    stateful=True # 상태유지 LSTM 모델. 여기서 상태유지라는 것은 현재 학습된 상태가 다음 학습시 초기 상태로 전달된다는 것을 의미
                   )                # 마지막 샘플 학습이 마치고, 새로운 에포크 수행 시에는 새로운 샘플 학습을 해야하므로 상태 초기화 필요
inf_decoder_outputs = inf_decoder_lstm(inf_decoder_inputs)
inf_decoder_dense = Dense(output_dim, activation='softmax') # 마지막 출력할 때는 소프트맥스 함수로 확률로 변환시킴
inf_decoder_outputs = inf_decoder_dense(inf_decoder_outputs)
sample_model = Model(inf_decoder_inputs, inf_decoder_outputs)


In [0]:
#Transfer Weights : 디코더 모델을 정의한 후 학습 된 자동 인코더 모델에서 해당 가중치가 전송됩니다.
for i in range(1,3):
    sample_model.layers[i].set_weights(model.layers[i+6].get_weights())
sample_model.save("Blog_simple_samplemodel.h5")

In [0]:
x_latent = smiles_to_latent_model.predict(X_test) # 잠재 모델에 대한 스마일은 잠재 공간과 같은 SMILES를 fingerprint로 인코딩하는데 사용할 수 있습니다.

In [0]:
# 비슷한 분자들은 비슷한 fingerprint를 만든다. 숨겨진 공간에서 비슷한 분자들이 비슷한 벡터들을 만드는 것을 보기 위해서 비슷한 분자들에 대한 간단한 검색의 수행된다. 
# 숨겨진 벡터간의 절대값의 차이는 행렬의 유사성을 이용한다. 이 테스트는 빠르게 수행될 수 있다.
# 여기서 분자 구조가 비슷한 분자들은 물성이 비슷하다는 과학적인 지식에서 출발한 개념이다.
molno = 5
latent_mol = smiles_to_latent_model.predict(X_test[molno:molno+1])
sorti = np.argsort(np.sum(np.abs(x_latent - latent_mol), axis=1))
print sorti[0:10]
print smiles_test.iloc[sorti[0:8]]
Draw.MolsToImage(smiles_test.iloc[sorti[0:8]].apply(Chem.MolFromSmiles))


In [0]:
# 위에서 구현한 내용을 그림으로 나타내 보면
Draw.MolsToImage(smiles_test.iloc[sorti[-8:]].apply(Chem.MolFromSmiles))

In [0]:
# 이온화되지 않은 화합물의 농도 비율을 logP라 한다. 분배계수(Partition-coefficient)는 섞이지 않는 2종류의 용매 혼합물에 화합물을 섞고 평형상태가 이루어졌을 때 각 용매에서의 화합물 농도 비율을 의미합니다.
# Octanol과 물에 대한 화합물의 분배계수를 통해 지질친화도를 평가할 수 있기 때문에 신약개발에서 중요하게 보는 물리화학적 성질입니다. 
# 농도 비율에 log를 취한 값을 보통 사용하는데, 이온화 되지 않은 화합물의 농도 비율을 logP라 합니다.
logp = smiles_test.apply(Chem.MolFromSmiles).apply(Descriptors.MolLogP)
# 사이킷런 import
from sklearn.decomposition import PCA  # PCA 를 이용해서 쉽게 차원을 축소시키고, 새로 만들어진 데이터로 dataframe 을 만들어줍니다.
pca = PCA(n_components = 2)
red = pca.fit_transform(x_latent)
plt.figure()
plt.scatter(red[:,0], red[:,1],marker='.', c= logp)
print(pca.explained_variance_ratio_, np.sum(pca.explained_variance_ratio_))


In [0]:
molwt = smiles_test.apply(Chem.MolFromSmiles).apply(Descriptors.MolMR)
plt.figure()
plt.scatter(red[:,0], red[:,1],marker='.', c= molwt)


In [0]:
#Model LogP?
x_train_latent = smiles_to_latent_model.predict(X_train)
logp_train = smiles_train.apply(Chem.MolFromSmiles).apply(Descriptors.MolLogP)

from keras.models import Sequential
logp_model = Sequential()
logp_model.add(Dense(128, input_shape=(latent_dim,), activation="relu"))
logp_model.add(Dense(128, activation="relu"))
logp_model.add(Dense(1))
logp_model.compile(optimizer="adam", loss="mse")

rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=10, min_lr=0.000001, verbose=1, min_delta=1e-5)
logp_model.fit(x_train_latent, logp_train, batch_size=128, epochs=400, callbacks = [rlr]) #callbacks : 정한 epoch 또는 1 epoch이 끝날때마다 이 함수를 불러서 정해놓은 기능

In [0]:
logp_pred_train = logp_model.predict(x_train_latent)
logp_pred_test = logp_model.predict(x_latent)
plt.scatter(logp, logp_pred_test, label="Test")
plt.scatter(logp_train, logp_pred_train, label="Train")
plt.legend()

In [0]:
#은닉 공간에서 smiles
def latent_to_smiles(latent):
    #decode states and set Reset the LSTM cells with them
    states = latent_to_states_model.predict(latent)
    sample_model.layers[1].reset_states(states=[states[0],states[1]])
    #Prepare the input char
    startidx = char_to_int["!"]
    samplevec = np.zeros((1,1,22))
    samplevec[0,0,startidx] = 1
    smiles = ""
    #Loop and predict next char
    for i in range(28):
        o = sample_model.predict(samplevec)
        sampleidx = np.argmax(o)
        samplechar = int_to_char[sampleidx]
        if samplechar != "E":
            smiles = smiles + int_to_char[sampleidx]
            samplevec = np.zeros((1,1,22))
            samplevec[0,0,sampleidx] = 1
        else:
            break
    return smiles

In [0]:
smiles = latent_to_smiles(x_latent[0:1])
print(smiles)
print(smiles_test.iloc[0])

In [0]:
wrong = 0
for i in range(1000):
    smiles = latent_to_smiles(x_latent[i:i+1])
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        pass
    else:
        print(smiles)
        wrong = wrong + 1
print ("%0.1F percent wrongly formatted smiles"%(wrong/float(1000)*100))

In [0]:
#Interpolation test in latent_space
i = 0
j= 2
latent1 = x_latent[j:j+1]
latent0 = x_latent[i:i+1]
mols1 = []
ratios = np.linspace(0,1,25)
for r in ratios:
    #print r
    rlatent = (1.0-r)*latent0 + r*latent1
    smiles  = latent_to_smiles(rlatent)
    mol = Chem.MolFromSmiles(smiles,sanitize=False)
    if mol:
        mols1.append(mol)
    else:
        print(smiles)
Draw.MolsToGridImage(mols1, molsPerRow=5)

In [0]:
#Sample around the latent vector 
latent = x_latent[0:1]
scale = 0.40
mols = []
for i in range(20):
    latent_r = latent + scale*(np.random.randn(latent.shape[1])) #TODO, try with
    smiles = latent_to_smiles(latent_r)
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        mols.append(mol)
    else:
        print(smiles)
Draw.MolsToGridImage(mols, molsPerRow=5)